## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-18-55-15 +0000,nyt,How the Heavy-Metal Fall of a Dictator Shapes ...,https://www.nytimes.com/2025/11/11/us/politics...
1,2025-11-11-18-51-50 +0000,nyt,"Turkey Seeks Jail Sentence of Over 2,000 Years...",https://www.nytimes.com/2025/11/11/world/middl...
2,2025-11-11-18-48-00 +0000,wsj,Comex Gold Settles 0.12% Lower at $4106.80,https://www.wsj.com/finance/commodities-future...
3,2025-11-11-18-47-14 +0000,nyt,Aircraft Carrier Moves into the Caribbean as U...,https://www.nytimes.com/2025/11/11/us/politics...
4,2025-11-11-18-44-55 +0000,nypost,Rare footage captures sucker fish catching a f...,https://nypost.com/2025/11/11/world-news/rare-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,40
153,shutdown,30
87,government,19
317,new,18
160,market,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
234,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,134
103,2025-11-11-15-24-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,116
303,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,114
210,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,107
66,2025-11-11-17-01-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
234,134,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
103,67,2025-11-11-15-24-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
41,53,2025-11-11-17-29-31 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
188,42,2025-11-11-08-43-05 +0000,wsj,"Stock Market Today: Dow Jumps, Nasdaq Slips; A...",https://www.wsj.com/livecoverage/stock-market-...
238,38,2025-11-11-00-52-53 +0000,nyt,Syria’s President Meets Trump at White House f...,https://www.nytimes.com/2025/11/10/world/middl...
7,37,2025-11-11-18-28-04 +0000,nypost,Woman killed by NYC subway train in apparent s...,https://nypost.com/2025/11/11/us-news/woman-ki...
304,36,2025-11-10-21-02-09 +0000,wapo,Immigrant detention firm seeks immunity in cas...,https://www.washingtonpost.com/business/2025/1...
102,31,2025-11-11-15-25-48 +0000,nypost,"Christian ministry CEO, daughter ID’d as victi...",https://nypost.com/2025/11/11/us-news/christia...
19,31,2025-11-11-18-10-28 +0000,nypost,US aircraft carrier moves into Latin America r...,https://nypost.com/2025/11/11/world-news/us-ai...
222,29,2025-11-11-02-12-03 +0000,nyt,Trump Threatens to Sue the BBC for $1 Billion ...,https://www.nytimes.com/2025/11/10/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
